In [ ]:
# 连接Google Drive
from google.colab import drive
import json
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


# Data Loading

Training Dataset

Data
The data are stored as a CSV and as a pickled pandas dataframe (Python 2.7). Each data file contains 5 columns:

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [ ]:
import pandas as pd
data=pd.read_csv('https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv')  
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In the modeling process, we use class as the target variable, tweet as input for now

# Modeling

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 8.9MB/s 
     |████████████████████████████████| 3.3MB 38.2MB/s 
     |████████████████████████████████| 870kB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=05e441c3a6257a1d4b2d472cc06db044f3729fce26088b4870359836e9891f5b
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import pickle
import numpy as np
import argparse
import numpy as np
from collections import Counter
import pdb
import time
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import datetime
from sklearn.metrics import confusion_matrix, classification_report
from transformers import BartTokenizer, BartForSequenceClassification

In [ ]:
!nvidia-smi

Thu Apr  8 19:40:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
sentences=data['tweet']
labels=data['class']
print("length of the dataset is ",len(labels))

length of the dataset is  24783


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=1)

In [ ]:
##Only needed for pkl file
def save_data(filename, data):
    #Storing data with labels
    a_file = open(filename, "wb")
    pickle.dump(data, a_file)
    a_file.close()
    
##Only needed for pkl file
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output

##Only needed for pkl file
def read_data(input_file, label_dict):
    data = load_data(input_file)

    sentences, labels = [], []
    
    for (id, sentence, label) in data:
        sentences.append(sentence)
        labels.append(label_dict[label])

    return sentences, labels

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def read_data_mult(input_file, label_dict,  original_labels, ratio):
    number = {0:0, 1:0, 2:0}
    for label in original_labels:
        number[label] += 1

    data = load_data(input_file)

    sentences, labels = [], []
    counter = {0:0, 1:0, 2:0}
    for (id, sentence, label) in data:
        label = label_dict[label]
        if counter[label] < int(number[label] * ratio):
            counter[label] += 1
            sentences.append(sentence)
            labels.append(label)

    return sentences, labels

In [ ]:
def tokenize_text(tokeinzer, sentences, labels):

    data_input_ids, data_attention_masks, output_labels = [], [], []

    #TODO: can this be optimized?
    max_num_tokens = 0
    for sent, label in zip(sentences, labels):
        tokenized_output = tokeinzer(sent)
        sent_input_ids = torch.tensor(tokenized_output['input_ids'])
        #print(sent_input_ids)
        sent_attention_mask = torch.tensor(tokenized_output['attention_mask'])
        #print(sent_attention_mask)
        #break
        if len(sent_input_ids) > max_num_tokens:
            max_num_tokens = len(sent_input_ids)

    for sent, label in zip(sentences, labels):
        tokenized_output = tokeinzer(sent, max_length=max_num_tokens, pad_to_max_length = True)
        sent_input_ids = torch.tensor(tokenized_output['input_ids'])
        sent_attention_mask = torch.tensor(tokenized_output['attention_mask'])
        data_input_ids.append(sent_input_ids)
        data_attention_masks.append(sent_attention_mask)
        output_labels.append(label)

    # Convert the lists into tensors.
    data_input_ids = torch.stack(data_input_ids, dim=0)
    data_attention_masks = torch.stack(data_attention_masks, dim=0)
    output_labels = torch.tensor(output_labels)

    return data_input_ids, data_attention_masks, output_labels

In [ ]:
def create_loader(tokenizer, sentences, labels, batch_size):

    input_ids, attention_masks, labels = tokenize_text(tokenizer, sentences, labels)

    dataset = TensorDataset(input_ids, attention_masks, labels)

    train_size = len(dataset)

    train_dataset = dataset

    print('{:>5,} training samples'.format(train_size))

    train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size 
        )

    return train_dataloader

In [ ]:
def train_epoch(train_dataloader, model, optimizer, scheduler):

    t0 = time.time()
    model.train()
    total_train_loss = 0

    correct = 0
    total = 0

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_size = b_labels.size(0)

        model.zero_grad()
        # pdb.set_trace()
        outputs = model(b_input_ids, 
                             #token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        total_train_loss += loss.item()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        smax = torch.softmax(logits, dim = 1)
        indices = torch.argmax(smax, dim = 1)
        correct += torch.sum(indices==b_labels).item()
        total += b_size


    avg_train_loss = total_train_loss / len(train_dataloader)  
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    val_accuracy = correct / total
    print(" Training Accuracy: {0:.4f}".format(val_accuracy))
    print('Time:', time.time() - t0)

In [ ]:
def eval_epoch(validation_dataloader, model):

    t0 = time.time()
    model.eval()

    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    for i, batch in enumerate(validation_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_size = b_labels.size(0)

        with torch.no_grad():
            outputs = model(b_input_ids, 
                                   #token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        loss = outputs.loss
        logits = outputs.logits
        smax = torch.softmax(logits, dim = 1)
        indices = torch.argmax(smax, dim = 1)

        correct += torch.sum(indices==b_labels).item()
        total += b_size

        #saving
        all_predictions += list(indices.cpu().numpy())
        all_labels += list(b_labels.cpu().numpy())

    print(classification_report(all_labels, all_predictions, target_names=['negative', 'neutral', 'positive'], digits = 5))
    #print(classification_report(all_labels, all_predictions, target_names=['negative', 'positive'], digits = 5))
    val_accuracy = correct / total
    print(" Validation Accuracy: {0:.4f}".format(val_accuracy))
    print('Time:', time.time() - t0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#model = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels = 3)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
#model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-base", num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)
#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
#model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 3)
model = model.to(device)

cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:



#for multilingual training
#sentences2, labels2 = read_data('sentiment_tamil_train.pkl', label_dict)
'''sentences2, labels2 = read_data_mult('sentiment_mal_train.pkl', label_dict,  original_labels = labels, ratio = 0.3)
print(len(sentences))
sentences += sentences2[:1000]
labels += labels2[:1000]
print(len(sentences))'''


print(Counter(labels))

Counter({1: 19190, 2: 4163, 0: 1430})


In [ ]:
batch_size = 16
num_epochs = 10
train_dataloader = create_loader(tokenizer, X_train, y_train, batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


19,826 training samples


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
#sentences, labels = read_data('sentiment_tamil_test.pkl', label_dict)
batch_size = 16
val_dataloader = create_loader(tokenizer, X_test, y_test, batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


4,957 training samples


In [ ]:
#eval_epoch(val_dataloader, model)
#print('End Zero Shot')
for i in range(num_epochs):
    #save_directory = 'saving' + str(i)
    #os.makedirs(save_directory, exist_ok=True)
    print(i)
    train_epoch(train_dataloader, model, optimizer, scheduler)
    #print ("\tSaving model at epoch: {}\t".format(i))
    #tokenizer.save_pretrained(save_directory)
    #model.save_pretrained(save_directory)

    #eval_epoch(val_dataloader, model)
    print()
    eval_epoch(val_dataloader, model)
    print('-'*20)
    #eval_epoch(test_dataloader2, model)
    #eval_epoch(test_dataloader3, model)
    print()
    print('='*20)
    #os.system('tar -zcvf ' + save_directory + '.tar.gz ' + save_directory)
    print()

0

  Average training loss: 0.28
  Training epcoh took: 0:16:45
 Training Accuracy: 0.8993
Time: 1004.989706993103

              precision    recall  f1-score   support

    negative    0.65385   0.05743   0.10559       296
     neutral    0.92321   0.97664   0.94917      3853
    positive    0.87018   0.92079   0.89477       808

    accuracy                        0.91265      4957
   macro avg    0.81574   0.65162   0.64984      4957
weighted avg    0.89848   0.91265   0.88993      4957

 Validation Accuracy: 0.9126
Time: 67.69392585754395
--------------------


1

  Average training loss: 0.20
  Training epcoh took: 0:16:45
 Training Accuracy: 0.9258
Time: 1005.4363224506378

              precision    recall  f1-score   support

    negative    0.59350   0.24662   0.34845       296
     neutral    0.93109   0.96782   0.94910      3853
    positive    0.87575   0.89851   0.88699       808

    accuracy                        0.91346      4957
   macro avg    0.80011   0.70432   0.

# New Section

In [ ]:
path="/content/gdrive/MyDrive/projectmodel_0408.pt"
torch.save(model.state_dict(), path)

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

In [ ]:
sentences, labels = read_data('sentiment_data_test.pkl', label_dict)
test_dataloader2 = create_loader(tokenizer, sentences, labels, batch_size)
eval_epoch(test_dataloader2, model)

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section